In [0]:
# ************** USE TO REMOVE DATABASE - DO NOT TOUCH **********
# ************** spark.sql("drop database warehousedb cascade")

Out[47]: DataFrame[]

In [0]:
# Create database
spark.sql("create database warehousedb1 comment 'Zac created this database using python based on tutorial by Cloudpandith: https://www.youtube.com/watch?v=dX_qS8vdLOs'")

Out[13]: DataFrame[]

In [0]:
# Display description of created database
df=spark.sql("desc database warehousedb1")
display(df)

database_description_item,database_description_value
Catalog Name,spark_catalog
Namespace Name,warehousedb1
Comment,Zac created this database using python based on tutorial by Cloudpandith: https://www.youtube.com/watch?v=dX_qS8vdLOs
Location,dbfs:/user/hive/warehouse/warehousedb1.db
Owner,root


In [0]:
# Use this code if needed to Drop Tables
# spark.sql("drop table warehousedb1.**TABLENAME**")

In [0]:
# Create Products Table - Note: KEY CONSTRAINTS not available for Databricks Community Tier - COMMENT used to represent KEY Constraints
spark.sql("""
    CREATE TABLE warehousedb1.Products(
        Part_No VARCHAR(30) COMMENT 'PRIMARY KEY',
        Category VARCHAR(30))
        """)

Out[15]: DataFrame[]

In [0]:
# Show description of created Table
df2=spark.sql("desc table warehousedb1.Products")
display(df2)

col_name,data_type,comment
Part_No,varchar(30),PRIMARY KEY
Category,varchar(30),null


In [0]:
# USE TO DROP TABLE warehousedb1.All_Sales
# ********* spark.sql("DROP TABLE warehousedb1.All_Sales")

Out[79]: DataFrame[]

In [0]:
# Create All_Sales Table in warehousedb1 database - Note: KEY CONSTRAINTS not available for Databricks Community Tier - COMMENT used to represent KEY Constraints
spark.sql("""
    CREATE TABLE warehousedb1.All_Sales(
          id BIGINT GENERATED ALWAYS AS IDENTITY COMMENT 'PRIMARY KEY',
          Part_No VARCHAR(30) COMMENT 'FOREIGN KEY >- warehousedb1.Products.Part_No',
          Invoice_Date date,
          Ship_Qty int,
          Extended_Price double,
          Margin_Percentage_Current double,
          Category string,
          Year int,
          Month int,
          Current_Cost double,
          Profit double
          )
""")

Out[17]: DataFrame[]

In [0]:
# Display Description of newly created All_Sales Table
df3=spark.sql("desc TABLE warehousedb1.All_Sales")
display(df3)

col_name,data_type,comment
id,bigint,PRIMARY KEY
Part_No,varchar(30),FOREIGN KEY >- warehousedb1.Products.Part_No
Invoice_Date,date,null
Ship_Qty,int,null
Extended_Price,double,null
Margin_Percentage_Current,double,null
Category,string,null
Year,int,null
Month,int,null
Current_Cost,double,null


In [0]:
# Query newly created warehousedb1.All_Sales Table
spark.sql("""
    SELECT *
    FROM warehousedb1.All_Sales
""").show()

+---+-------------+------------+--------+--------------+-------------------------+--------------------+----+-----+------------------+------------------+
| id|      Part_No|Invoice_Date|Ship_Qty|Extended_Price|Margin_Percentage_Current|            Category|Year|Month|      Current_Cost|            Profit|
+---+-------------+------------+--------+--------------+-------------------------+--------------------+----+-----+------------------+------------------+
|  1|123HarPVC20LV|  2018-01-02|       1|         33.95|                     38.0|       PVC Chair Mat|2018|    1|            21.049|12.901000000000002|
|  3|FPRPreEnt72CH|  2018-01-04|       1|         101.9|                     58.0|        Entrance Mat|2018|    1| 42.79800000000001|            59.102|
|  5| FCOCoAnt36BK|  2018-01-09|       2|         100.4|                     58.0|    Anti-Fatigue Mat|2018|    1|42.168000000000006|            58.232|
|  7|111AllPol23ER|  2018-01-11|       2|         260.0|                     45.0|

In [0]:
# Create SARIMAX Table
# Following document used to create 'id' column that generates unique values as data is inserted into table: https://www.databricks.com/blog/2022/08/08/identity-columns-to-generate-surrogate-keys-are-now-available-in-a-lakehouse-near-you.html
spark.sql("""
    CREATE TABLE warehousedb1.SARIMAX(
          id BIGINT GENERATED ALWAYS AS IDENTITY COMMENT 'PRIMARY KEY',
          Category string COMMENT 'FOREIGN KEY >- warehousedb1.Products.Category',
          Predicted_Sales_Qty FLOAT,
          Margin_Error FLOAT,
          AIC_Score FLOAT,
          Error FLOAT,
          Year int,
          Month int     
          )
          """)

Out[20]: DataFrame[]

In [0]:
# Display description of newly created SARIMAX Table
df4=spark.sql("desc TABLE warehousedb1.SARIMAX")
display(df4)

col_name,data_type,comment
id,bigint,PRIMARY KEY
Category,string,FOREIGN KEY >- warehousedb1.Products.Category
Predicted_Sales_Qty,float,null
Margin_Error,float,null
AIC_Score,float,null
Error,float,null
Year,int,null
Month,int,null


In [0]:
# Query Newly Created SARIMAX Table
spark.sql("""
    SELECT *
    FROM warehousedb1.SARIMAX
    """).show()

+---+----------------+-------------------+------------+---------+--------+----+-----+
| id|        Category|Predicted_Sales_Qty|Margin_Error|AIC_Score|   Error|Year|Month|
+---+----------------+-------------------+------------+---------+--------+----+-----+
|  1|  All categories|          1935.1199|   687.87555|697.15643|12.28205|2023|    1|
|  7|  All categories|          1677.1814|    726.3752|697.15643|12.28205|2023|    2|
| 13|  All categories|          1840.5173|   762.93445|697.15643|12.28205|2023|    3|
| 19|  All categories|           1774.984|    797.8201|697.15643|12.28205|2023|    4|
| 25|  All categories|          1859.6893|    831.2431|697.15643|12.28205|2023|    5|
| 31|  All categories|           1800.217|    863.3731|697.15643|12.28205|2023|    6|
| 37|  All categories|            1734.61|    894.3496|697.15643|12.28205|2023|    7|
| 43|  All categories|          1889.2223|    924.2885|697.15643|12.28205|2023|    8|
| 49|  All categories|          1838.1172|   953.28766

In [0]:
# Create Predictions Table
# Following document used to create 'id' column that generates unique values as data is inserted into table: https://www.databricks.com/blog/2022/08/08/identity-columns-to-generate-surrogate-keys-are-now-available-in-a-lakehouse-near-you.html
spark.sql("""
    CREATE TABLE warehousedb1.Predictions(
        id BIGINT GENERATED ALWAYS AS IDENTITY COMMENT 'PRIMARY KEY',
        Category string COMMENT 'FOREIGN KEY >- warehousedb1.Products.Category',
        One_Month_Sales FLOAT,
        Two_Month_Sales FLOAT,
        Three_Month_Sales FLOAT,
        Four_Month_Sales FLOAT,
        Five_Month_Sales FLOAT,
        Six_Month_Sales FLOAT,
        Seven_Month_Sales FLOAT,
        Eight_Month_Sales FLOAT,
        Nine_Month_Sales FLOAT,
        Ten_Month_Sales FLOAT,
        Eleven_Month_Sales FLOAT,
        Twelve_Month_Sales FLOAT
        )
""")

Out[23]: DataFrame[]

In [0]:
# Display description of warehousedb1.Predictions Table
df5=spark.sql("desc TABLE warehousedb1.Predictions")
display(df5)

col_name,data_type,comment
id,bigint,PRIMARY KEY
Category,string,FOREIGN KEY >- warehousedb1.Products.Category
One_Month_Sales,float,null
Two_Month_Sales,float,null
Three_Month_Sales,float,null
Four_Month_Sales,float,null
Five_Month_Sales,float,null
Six_Month_Sales,float,null
Seven_Month_Sales,float,null
Eight_Month_Sales,float,null


In [0]:
# Query warehousedb1.Predictions Table
spark.sql("""
    SELECT *
    FROM warehousedb1.Predictions
""").show()

+---+--------+---------------+---------------+-----------------+----------------+----------------+---------------+-----------------+-----------------+----------------+---------------+------------------+------------------+
| id|Category|One_Month_Sales|Two_Month_Sales|Three_Month_Sales|Four_Month_Sales|Five_Month_Sales|Six_Month_Sales|Seven_Month_Sales|Eight_Month_Sales|Nine_Month_Sales|Ten_Month_Sales|Eleven_Month_Sales|Twelve_Month_Sales|
+---+--------+---------------+---------------+-----------------+----------------+----------------+---------------+-----------------+-----------------+----------------+---------------+------------------+------------------+
+---+--------+---------------+---------------+-----------------+----------------+----------------+---------------+-----------------+-----------------+----------------+---------------+------------------+------------------+



In [0]:
# Create Purchases Table
spark.sql("""
    CREATE TABLE warehousedb1.Purchases(
          id BIGINT GENERATED ALWAYS AS IDENTITY COMMENT 'PRIMARY KEY',
          Part_No VARCHAR(30) COMMENT 'FOREIGN KEY >- warehousedb1.Products.Part_No',
          Received_Date DATE,
          Qty_Received INT,
          Base_Unit_Price DOUBLE,
          Year INT,
          Month INT,
          Category VARCHAR(30) COMMENT 'FOREIGN KEY >- Products.Category'
          )
""")

Out[26]: DataFrame[]

In [0]:
# Display Description for warehousedb1.Purchases Table
df6=spark.sql("desc TABLE warehousedb1.Purchases")
display(df6)

col_name,data_type,comment
id,bigint,PRIMARY KEY
Part_No,varchar(30),FOREIGN KEY >- warehousedb1.Products.Part_No
Received_Date,date,null
Qty_Received,int,null
Base_Unit_Price,double,null
Year,int,null
Month,int,null
Category,varchar(30),FOREIGN KEY >- Products.Category


In [0]:
# Query warehousedb1.Purchases Table
spark.sql("""
    SELECT *
    FROM warehousedb1.Purchases
""").show()

+---+-------+-------------+------------+---------------+----+-----+--------+
| id|Part_No|Received_Date|Qty_Received|Base_Unit_Price|Year|Month|Category|
+---+-------+-------------+------------+---------------+----+-----+--------+
+---+-------+-------------+------------+---------------+----+-----+--------+

